In [10]:
import os
import re
from typing import List

import docx2txt
import matplotlib.pyplot as plt
from openai import OpenAI
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

In [11]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv(filename="openai.env"), override=True)

True

In [ ]:
client = OpenAI()

In [12]:
data_path_business = "../../resources/sentenze_idealtipiche/pro_business"
data_path_labour = "../../resources/sentenze_idealtipiche/pro_labour"

texts_list = []

for filename_business in os.listdir(data_path_business):
    file_path_business = os.path.join(data_path_business, filename_business)
    text_business = docx2txt.process(file_path_business)
    texts_list.append(text_business)

for filename_labour in os.listdir(data_path_labour):
    file_path_labour = os.path.join(data_path_labour, filename_labour)
    text_labour = docx2txt.process(file_path_labour)
    texts_list.append(text_labour)

texts_list

{'PRO_BUSINESS - TMI12_4_12 Fiom Fiat costituzione RSA senza firma contratto collettivo.docx': 'Tribunale Milano (decreto ex art. 28, legge 300/1970) – 12 aprile 2012 – Est. Taraborrelli – Fiom c. Sirio S.c.p.A.\n\n\n\nStatuto dei Lavoratori – Rappresentanze sindacali aziendali - Art. 19, legge n. 300 del 1970 – Requisito per la costituzione – Sindacati firmatari di contratto collettivo applicato nell’unità poduttiva – Difetto del requisito – Diritto a costituire RSA – Esclusione.\n\n\n\nL\'estromissione di un sindacato, per quanto rilevante e maggioritario in azienda e, quindi, rappresentativo ma non contraente di contratto collettivo applicato ex art. 19, legge n. 300 del 1970, non può essere sanzionato quando avviene a seguito di comportamento unilaterale ma ammesso in ossequio alla legge, già costituzionalmente ritenuta valida, che attribuisce i diritti di cui al citato art. 19 ai soli sindacati firmatari di contratto collettivo applicato in azienda.(*)\n\n\n\n\n\n\n\nla domanda di

In [13]:
def clean_texts(texts: List[str]) -> List[str]:
    cleaned_texts = []
    for sentence in texts:
        clean_text = sentence.replace('\n', ' ')
        clean_text = clean_text.replace("\\", "")
        clean_text = clean_text.replace('\"', '')
        clean_text = clean_text.replace('\\"', '')
        clean_text = clean_text.replace(r"\'", "'")
        clean_text = clean_text.replace(r'\"', '')
        clean_text = clean_text.replace(r'\\"', '')
        text_splitted = re.findall('(.*?)(PQM|P.Q.M.|P. Q. M.)', clean_text)
        if text_splitted:
            clean_text = ' '.join(text_splitted[0])
            clean_text = re.sub(r'\s+', ' ', clean_text)
            clean_text = re.sub(r'(PQM|P.Q.M.|P. Q. M.)', '', clean_text)
            clean_text = clean_text.strip()
        cleaned_texts.append(clean_text)
    return cleaned_texts

In [14]:
cleaned_texts = clean_texts(texts=texts_list)

In [15]:
cleaned_texts

{'PRO_BUSINESS - TMI12_4_12 Fiom Fiat costituzione RSA senza firma contratto collettivo.docx': "Tribunale Milano (decreto ex art. 28, legge 300/1970) – 12 aprile 2012 – Est. Taraborrelli – Fiom c. Sirio S.c.p.A.    Statuto dei Lavoratori – Rappresentanze sindacali aziendali - Art. 19, legge n. 300 del 1970 – Requisito per la costituzione – Sindacati firmatari di contratto collettivo applicato nell’unità poduttiva – Difetto del requisito – Diritto a costituire RSA – Esclusione.    L'estromissione di un sindacato, per quanto rilevante e maggioritario in azienda e, quindi, rappresentativo ma non contraente di contratto collettivo applicato ex art. 19, legge n. 300 del 1970, non può essere sanzionato quando avviene a seguito di comportamento unilaterale ma ammesso in ossequio alla legge, già costituzionalmente ritenuta valida, che attribuisce i diritti di cui al citato art. 19 ai soli sindacati firmatari di contratto collettivo applicato in azienda.(*)        la domanda di repressione di c

In [ ]:
def process_embeddings(embeddings: List) -> List:
    embeddings_post = []
    for embed in embeddings:
        embeddings_post.append(embed.embedding)
    return embeddings_post

In [ ]:
def get_embeddings_openai(texts_list: List[str]) -> List:
    embeddings = client.embeddings.create(
        input=texts_list,
        model="text-embedding-ada-002",
    ).data
    embeddings_post = process_embeddings(embeddings=embeddings)
    return embeddings_post

In [ ]:
def clustering_embeddings(n_clusters, embeddings):
    # Esegui il clustering K-means
    kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(embeddings)
    labels = kmeans.labels_

    # Esegui la PCA per ridurre la dimensionalità a 2
    pca = PCA(n_components=2)
    reduced_embeddings = pca.fit_transform(embeddings)

    # Assegna le etichette "A" e "B" ai cluster
    label_map = {0: "Lavoratore", 1: "Datore di lavoro"}
    mapped_labels = [label_map[label] for label in labels]

    # Visualizza i cluster in uno spazio bidimensionale
    fig, ax = plt.subplots()
    for i, (coords, label) in enumerate(zip(reduced_embeddings, mapped_labels)):
        ax.scatter(coords[0], coords[1], label=label)
        ax.text(coords[0], coords[1], f"Testo {i + 1}")

    plt.xlabel("PC1")
    plt.ylabel("PC2")
    plt.title("Visualizzazione dei cluster")
    plt.show()

    # Unisci le coordinate ridotte e le etichette
    # result = list(zip(reduced_embeddings, mapped_labels))
    #
    # return result

In [ ]:
embeddings = get_embeddings_openai(texts_list=cleaned_texts)

In [ ]:
clustering_embeddings(n_clusters=2, embeddings=embeddings)